## 12.1 json 파일 읽고 쓰기

In [1]:
#json의 형태는 "키":"값" 형식
#키는 문자열이어야 한다.



12.1.1 json파일을 DataFrame으로 불러오기

In [2]:
import pandas as pd

In [3]:
file='../DATA/nobel.json'

In [4]:
nobel=pd.read_json(file)
nobel

,prizes
0,"{'year': '2019', 'category': 'chemistry', 'lau..."
1,"{'year': '2019', 'category': 'economics', 'lau..."
2,"{'year': '2019', 'category': 'literature', 'la..."
3,"{'year': '2019', 'category': 'peace', 'laureat..."
4,"{'year': '2019', 'category': 'physics', 'overa..."
...,...
641,"{'year': '1901', 'category': 'chemistry', 'lau..."
642,"{'year': '1901', 'category': 'literature', 'la..."
643,"{'year': '1901', 'category': 'peace', 'laureat..."
644,"{'year': '1901', 'category': 'physics', 'laure..."


In [5]:
nobel.loc[2,"prizes"]

{'year': '2019',
 'category': 'literature',
 'laureates': [{'id': '980',
   'firstname': 'Peter',
   'surname': 'Handke',
   'motivation': '"for an influential work that with linguistic ingenuity has explored the periphery and the specificity of human experience"',
   'share': '1'}]}

In [6]:
type(nobel.loc[2,"prizes"]) #행이 딕셔너리 시리즈임

dict

In [7]:
# 실습 목표는 데이터를 표로 변환하는 것.... => 키 값 year,category를 추출하여 DataFrame 열을 분리해야 한다.



## [평탄화=정규화] 실습

In [8]:
# 중첩된 데이터 레코드를 단일의 1차원 리스트로 변형하는 것이 이 과정의 목적이다.

In [9]:
#json_normalize 함수를 사용하자.


In [10]:
chemistry_2019=nobel.loc[0,"prizes"]
chemistry_2019

{'year': '2019',
 'category': 'chemistry',
 'laureates': [{'id': '976',
   'firstname': 'John',
   'surname': 'Goodenough',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '977',
   'firstname': 'M. Stanley',
   'surname': 'Whittingham',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'},
  {'id': '978',
   'firstname': 'Akira',
   'surname': 'Yoshino',
   'motivation': '"for the development of lithium-ion batteries"',
   'share': '3'}]}

In [11]:
pd.json_normalize(data=chemistry_2019)  #정규화 되었다.....

,year,category,laureates
0,2019,chemistry,"[{'id': '976', 'firstname': 'John', 'surname':..."


In [12]:
#그 다음 문자열을 매개변수로 전달

In [13]:
pd.json_normalize(data=chemistry_2019,record_path="laureates") #laureates" 가 정규화 됨

,id,firstname,surname,motivation,share
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3


근데 기존의 year랑 category가 사라짐....

In [14]:
pd.json_normalize(
    data=chemistry_2019,
    record_path="laureates",
    meta=["year","category"])  #이름을 담은 리스트[]를 meta라는 매개변수에 저장

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry


In [15]:
#  pd.json_normalize(
# data=nobel["prizes"],
# record_path="laureates",
# meta=["year","category"]
# )   

### 오류가 뜬다... 딕셔너리 중 일부에 laureates 키가 없다...

In [16]:
# 수동으로 키를 할당 해보자

In [17]:
cheese_consumption={
    "France":57.9,
    "Germany":53.2,
    "Luxembourg":53.2
}

In [18]:
cheese_consumption.setdefault("France",100)

57.9

In [19]:
cheese_consumption.setdefault("Italy",48)

48

In [20]:
def add_laureates_key(entry): #함수 정의(매개변수 :entry)
    entry.setdefault("laureates",[]) # laureates에 빈 리스트를 값으로 갖도록 키를 추가

nobel["prizes"].apply(add_laureates_key) 

0      None
1      None
2      None
3      None
4      None
       ... 
641    None
642    None
643    None
644    None
645    None
Name: prizes, Length: 646, dtype: object

In [21]:
#setdefault는 딕셔너리 자체를 변경하므로 기존의 시리즈를 덮어쓸 필요가 없음....

In [22]:
# meta매개변수로 딕셔너리 키 전달

In [23]:
winners=pd.json_normalize(
    data=nobel["prizes"],
    record_path="laureates",
    meta=["year","category"]
)


In [24]:
winners

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry
3,982,Abhijit,Banerjee,"""for their experimental approach to alleviatin...",3,2019,economics
4,983,Esther,Duflo,"""for their experimental approach to alleviatin...",3,2019,economics
...,...,...,...,...,...,...,...
945,569,Sully,Prudhomme,"""in special recognition of his poetic composit...",1,1901,literature
946,462,Henry,Dunant,"""for his humanitarian efforts to help wounded ...",2,1901,peace
947,463,Frédéric,Passy,"""for his lifelong work for international peace...",2,1901,peace
948,1,Wilhelm Conrad,Röntgen,"""in recognition of the extraordinary services ...",1,1901,physics


12.1.2 DataFrame을 json파일로 내보내기

In [25]:
#to_join메서드 사용

In [26]:
winners.head(2).to_json(orient='records')

'[{"id":"976","firstname":"John","surname":"Goodenough","motivation":"\\"for the development of lithium-ion batteries\\"","share":"3","year":"2019","category":"chemistry"},{"id":"977","firstname":"M. Stanley","surname":"Whittingham","motivation":"\\"for the development of lithium-ion batteries\\"","share":"3","year":"2019","category":"chemistry"}]'

In [27]:
# 열 이름 방지 하기...


In [28]:
winners.head(2).to_json(orient='split')

'{"columns":["id","firstname","surname","motivation","share","year","category"],"index":[0,1],"data":[["976","John","Goodenough","\\"for the development of lithium-ion batteries\\"","3","2019","chemistry"],["977","M. Stanley","Whittingham","\\"for the development of lithium-ion batteries\\"","3","2019","chemistry"]]}'

In [29]:
winners.to_json("winners.json",orient='records')

12.2 CSV 파일 읽고 쓰기

In [30]:
file2="../DATA/happiness.csv"

In [31]:
happyDF=pd.read_csv(file2)

In [32]:
happyDF

,Country,Score,GDP per capita,Social support,Life expectancy,Generosity
0,Finland,7.769,1.340,1.587,0.986,0.153
1,Denmark,7.600,1.383,1.573,0.996,0.252
2,Norway,7.554,1.488,1.582,1.028,0.271
3,Iceland,7.494,1.380,1.624,1.026,0.354
4,Netherlands,7.488,1.396,1.522,0.999,0.322
...,...,...,...,...,...,...
151,Rwanda,3.334,0.359,0.711,0.614,0.217
152,Tanzania,3.231,0.476,0.885,0.499,0.276
153,Afghanistan,3.203,0.350,0.517,0.361,0.158
154,Central African Republic,3.083,0.026,0.000,0.105,0.235


In [33]:
happyDF.head(10).to_csv()

',Country,Score,GDP per capita,Social support,Life expectancy,Generosity\r\n0,Finland,7.769,1.34,1.587,0.986,0.153\r\n1,Denmark,7.6,1.383,1.573,0.996,0.252\r\n2,Norway,7.554,1.488,1.582,1.028,0.271\r\n3,Iceland,7.494,1.38,1.624,1.026,0.354\r\n4,Netherlands,7.488,1.396,1.522,0.999,0.322\r\n5,Switzerland,7.48,1.452,1.526,1.052,0.263\r\n6,Sweden,7.343,1.387,1.487,1.009,0.267\r\n7,New Zealand,7.307,1.303,1.557,1.026,0.33\r\n8,Canada,7.278,1.365,1.505,1.039,0.285\r\n9,Austria,7.246,1.376,1.475,1.016,0.244\r\n'

12.3 엑셀 통합문서에 읽고 쓰기

12.3.2 엑셀 통합문서 가져오기

In [34]:
file2='../DATA/Single Worksheet.xlsx'

In [35]:
SingleDF=pd.read_excel(file2)

In [36]:
SingleDF

,First Name,Last Name,City,Gender
0,Brandon,James,Miami,M
1,Sean,Hawkins,Denver,M
2,Judy,Day,Los Angeles,F
3,Ashley,Ruiz,San Francisco,F
4,Stephanie,Gomez,Portland,F


In [37]:
pd.read_excel(io=file2,usecols=["City","First Name","Last Name"],   #city열을 인덱스로.....
              index_col="City")

,First Name,Last Name
City,,
Miami,Brandon,James
Denver,Sean,Hawkins
Los Angeles,Judy,Day
San Francisco,Ashley,Ruiz
Portland,Stephanie,Gomez


In [38]:
#기본적으로 판다스는 첫번쨰 시트만 가져오는데 sheet_name 매개변수에 인덱스 위치나 이름을 저장할 수 있다.

In [39]:
# pd.read_excel(file2,sheet_name=0)
# pd.read_excel(file2,sheet_name="Data 1")



In [40]:
# pd.read_excel(
#     file2,sheet_name=["Data 1","Data 3"] )

12.4 코딩 챌린지

In [41]:
file3='../DATA/tv_shows.json'

In [42]:
tv_shows_json=pd.read_json(file3)

In [43]:
tv_shows_json

,shows
0,"{'show': 'The X-Files', 'runtime': 60, 'networ..."
1,"{'show': 'Lost', 'runtime': 60, 'network': 'AB..."
2,"{'show': 'Buffy the Vampire Slayer', 'runtime'..."


1. shows열의 각 딕셔너리에 중첩된 에피소드 데이터를 정규화. 각 에피소드를 별도의 행으로 나타내는 DataFrmae 생성

In [45]:
tv_shows=pd.json_normalize(
data=tv_shows_json["shows"], #열의 데이터가 정규화
record_path= "episodes",   #에피소드와 관련된
meta=["show","runtime","network"] # 최상위 정보는 메타에 저장
)

In [46]:
tv_shows

,season,episode,name,air_date,show,runtime,network
0,1,1,Pilot,1993-09-11 01:00:00,The X-Files,60,FOX
1,1,2,Deep Throat,1993-09-18 01:00:00,The X-Files,60,FOX
2,1,3,Squeeze,1993-09-25 01:00:00,The X-Files,60,FOX
3,1,4,Conduit,1993-10-02 01:00:00,The X-Files,60,FOX
4,1,5,The Jersey Devil,1993-10-09 01:00:00,The X-Files,60,FOX
...,...,...,...,...,...,...,...
477,7,18,Dirty Girls,2003-04-16 00:00:00,Buffy the Vampire Slayer,60,UPN
478,7,19,Empty Places,2003-04-30 00:00:00,Buffy the Vampire Slayer,60,UPN
479,7,20,Touched,2003-05-07 00:00:00,Buffy the Vampire Slayer,60,UPN
480,7,21,End of Days,2003-05-14 00:00:00,Buffy the Vampire Slayer,60,UPN


2. 정규화된 데이터 셋을 각 프로그램 별로 하나씩 총 3개의 개별 DataFrame으로 필터링

In [48]:
xfiles=tv_shows[tv_shows["show"]=="The X-Files"]
lost=tv_shows[tv_shows["show"]=="Lost"]
buffy=tv_shows[tv_shows["show"]=="Buffy the Vampire Slayer"]   

In [50]:
print(xfiles,lost,buffy,sep='\n\n')

     season  episode                   name             air_date         show  \
0         1        1                  Pilot  1993-09-11 01:00:00  The X-Files   
1         1        2            Deep Throat  1993-09-18 01:00:00  The X-Files   
2         1        3                Squeeze  1993-09-25 01:00:00  The X-Files   
3         1        4                Conduit  1993-10-02 01:00:00  The X-Files   
4         1        5       The Jersey Devil  1993-10-09 01:00:00  The X-Files   
..      ...      ...                    ...                  ...          ...   
213      11        6                 Kitten  2018-02-08 01:00:00  The X-Files   
214      11        7           Rm9sbG93ZXJz  2018-03-01 01:00:00  The X-Files   
215      11        8               Familiar  2018-03-08 01:00:00  The X-Files   
216      11        9  Nothing Lasts Forever  2018-03-15 00:00:00  The X-Files   
217      11       10         My Struggle IV  2018-03-22 00:00:00  The X-Files   

    runtime network  
0    

3. episodes.xlsx엑셀 통합문서로 3개의 데이터프레임을 작성하고 각 프로그램의 에피소드 데이터를 별도의 워크시트에 저장...

1. 먼저 excelwriter 객체를 변수에 저장

In [51]:
episodes=pd.ExcelWriter("episodes.xlsx")
episodes

In [53]:
xfiles.to_excel(
    excel_writer=episodes,sheet_name="X-Flies",index=False  #각각 저장해주기(to_excel 사용)
)

lost.to_excel(
    excel_writer=episodes,sheet_name="Lost",index=False
)

buffy.to_excel(
    excel_writer=episodes,sheet_name="Buffy the Vampire Slayer",index=False
)

In [54]:
#index=False의 의미: DATAframe 인덱스를 제외하기 위해